In [15]:
import sklearn
from sklearn.datasets import load_files
import pandas as pd

In [24]:
movie_data = pd.read_csv (r'movie_reviews.txt')

#Change the txt into a csv file
movie_data.to_csv (r'movie.csv', index=None)

In [25]:
movie_data.head()

,Summary,Sentiment
0,rock destined st century new conan going make ...,1
1,gorgeously elaborate continuation lord ring tr...,1
2,effective tepid biopic,1
3,sometimes like go movie fun wasabi good place ...,1
4,emerges something rare issue movie honest keen...,1


In [26]:
#Set X as movie_data[‘Summary’] and y as movie_data[‘Sentiment’]
X = movie_data['Summary']
y = movie_data['Sentiment']

In [27]:
from sklearn.model_selection import train_test_split
docs_train, docs_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 12)

In [31]:
# initialize CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer


movieVzer = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize)         # use all 25K words. Higher accuracy

# fit and tranform using training text 
docs_train_counts = movieVzer.fit_transform(docs_train)

In [37]:
# finding screen
movieVzer.vocabulary_.get('screen')

6152

In [78]:
# Finding Steven Seagal
movieVzer.vocabulary_.get('Steven Seagal')

In [79]:
# Determine the shape of ‘docs_train_counts
docs_train_counts.shape

(8529, 7993)

In [44]:
# Convert raw frequency counts into TF-IDF values
from sklearn.feature_extraction.text import TfidfTransformer
movieTfmer = TfidfTransformer()
docs_train_tfidf = movieTfmer.fit_transform(docs_train_counts)

In [45]:
# Same dimensions, now with tf-idf values instead of raw frequency counts
docs_train_tfidf.shape

(8529, 7993)

In [46]:
# Using the fitted vectorizer and transformer, tranform the test data
docs_test_counts = movieVzer.transform(docs_test)
docs_test_tfidf = movieTfmer.transform(docs_test_counts)


In [47]:
# Now ready to build a classifier. 
# using Multinominal Naive Bayes as our model
from sklearn.naive_bayes import MultinomialNB

In [48]:
# Now ready to build a classifier. 
# We will use Multinominal Naive Bayes as our model
from sklearn.naive_bayes import MultinomialNB
# Train a Multinomial Naive Bayes classifier. Again, we call it "fitting"
clf = MultinomialNB()
clf.fit(docs_train_tfidf, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [49]:
# Predict the Test set results, find accuracy
y_pred = clf.predict(docs_test_tfidf)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.76699484294421

In [75]:
# Making the Confusion Matrix & classification_report
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm

array([[805, 271],
       [226, 831]], dtype=int64)

In [77]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.75      0.76      1076
           1       0.75      0.79      0.77      1057

    accuracy                           0.77      2133
   macro avg       0.77      0.77      0.77      2133
weighted avg       0.77      0.77      0.77      2133



In [73]:
#Use GridSearchCV with a logistic regression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
parameter_grid = {'C': [0.01, 0.1, 1, 2, 10, 100], 'penalty': ['l1', 'l2']}
clf = GridSearchCV(LogisticRegression(solver='liblinear'),parameter_grid, cv=5, scoring="roc_auc")
clf.fit(docs_train_tfidf, y_train)
y_pred_acc = clf.predict(docs_test_tfidf)

print("accuracy :" + str(accuracy_score(y_test,y_pred_acc)))
print("Best cross-validation score: {:.2f}".format(clf.best_score_))
print("Best parameters: ", clf.best_params_)
print("Best estimator: ", clf.best_estimator_)


C:\Users\Prashant Mourya\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_folds = np.zeros(n_samples, dtype=np.int)
C:\Users\Prashant Mourya\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:442: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. I

accuracy :0.7580872011251758
Best cross-validation score: 0.83
Best parameters:  {'C': 2, 'penalty': 'l2'}
Best estimator:  LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)


C:\Users\Prashant Mourya\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:794: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
C:\Users\Prashant Mourya\Anaconda3\lib\site-packages\sklearn\linear_model\base.py:291: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use

In [70]:
# Making the Confusion Matrix 
confusion_matrix(y_test,y_pred_acc)



array([[802, 274],
       [242, 815]], dtype=int64)